<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/jopara_token_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers datasets evaluate accelerate wandb
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Token classification

In [2]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/wVHdVlPScxA?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Token classification assigns a label to individual tokens in a sentence. One of the most common token classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

This guide will show you how to:

1. Finetune [DistilBERT](https://huggingface.co/distilbert/distilbert-base-uncased) on the [WNUT 17](https://huggingface.co/datasets/wnut_17) dataset to detect new entities.
2. Use your finetuned model for inference.

<Tip>

To see all architectures and checkpoints compatible with this task, we recommend checking the [task-page](https://huggingface.co/tasks/token-classification).

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate seqeval
```

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Load WNUT 17 dataset

Start by loading the WNUT 17 dataset from the 🤗 Datasets library:

In [4]:
#from datasets import load_dataset

#wnut = load_dataset("wnut_17", data_dir="wnut_17", revision="refs/convert/parquet")
#gn = load_dataset("unimelb-nlp/wikiann", "gn")
#es = load_dataset("unimelb-nlp/wikiann", "es")

# Download from github: GUA-SPA
# clone repo
! git clone https://github.com/pln-fing-udelar/gua-spa-2023.git
! pip install conllu

fatal: destination path 'gua-spa-2023' already exists and is not an empty directory.


In [87]:
import conllu

def read_conllu_file(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        data = f.read()
    return conllu.parse(data)

def conllu_to_dataset_format(parsed_data):
    dataset_format = {"text": [], "tokens": [], "ner_tags": []}
    for sentence in parsed_data:
        text = []
        tokens = []
        ner_tags = []
        last = ""
        for token in sentence:
            tokens.append(token["form"])
            if token['lemma'] is not None: # to IOB format
                ner_tag=token['lemma'].split('-')
                if last == ner_tag[0]:
                    prefix = "I"
                else:
                    prefix = "B"
                last = ner_tag[0]
                # final tags
                if 'ne' in ner_tag:
                    ner_tag = f"{ner_tag[1].upper()}-{ner_tag[2].upper()}"
                elif 'other' in ner_tag:
                    ner_tag = "O"
                else:
                    ner_tag = f"{prefix}-{ner_tag[0].upper()}"
            ner_tags.append(ner_tag)
        dataset_format["text"].append(" ".join(tokens))
        dataset_format["tokens"].append(tokens)
        dataset_format["ner_tags"].append(ner_tags)
    return dataset_format


In [88]:
# Read an example conllu file
train_data = read_conllu_file("gua-spa-2023/gua_spa_train.txt")
# Convert to dataset format
train_dataset_format = conllu_to_dataset_format(train_data)
# Print the first example
print(train_dataset_format["text"][0])
print(train_dataset_format["tokens"][0])
print(train_dataset_format["ner_tags"][0])

Aldana he'íva umi kits ohóva ha oguahëva opavave temimbo'épe oñepyrû mboyve clase pero noñeguahëi mbohapýha ary ohóvape .
['Aldana', "he'íva", 'umi', 'kits', 'ohóva', 'ha', 'oguahëva', 'opavave', "temimbo'épe", 'oñepyrû', 'mboyve', 'clase', 'pero', 'noñeguahëi', 'mbohapýha', 'ary', 'ohóvape', '.']
['B-PER', 'B-GN', 'I-GN', 'B-FOREIGN', 'B-GN', 'I-GN', 'I-GN', 'I-GN', 'I-GN', 'I-GN', 'I-GN', 'B-ES', 'I-ES', 'B-GN', 'I-GN', 'I-GN', 'I-GN', 'O']


In [89]:
# Read an example conllu file
dev_data = read_conllu_file("gua-spa-2023/gua_spa_dev_gold.txt")
# Convert to dataset format
dev_dataset_format = conllu_to_dataset_format(dev_data)
# Print the first example
print(dev_dataset_format["tokens"][0])
print(dev_dataset_format["ner_tags"][0])

['Obligarle', 'alguien', 'pa', 'que', 'me', 'escriba', '?', 'No', 'señorito', 'ani', 'nde', 'kangy']
['B-ES', 'I-ES', 'I-ES', 'I-ES', 'I-ES', 'I-ES', 'O', 'B-ES', 'I-ES', 'B-GN', 'I-GN', 'I-GN']


In [90]:
# Read an example conllu file
test_data = read_conllu_file("gua-spa-2023/gua_spa_test_gold.txt")
# Convert to dataset format
test_dataset_format = conllu_to_dataset_format(test_data)
# Print the first example
print(test_dataset_format["tokens"][0])
print(test_dataset_format["ner_tags"][0])

['Igusto', "ñañe'ẽ", 'guaranime', 'ha', 'avei', 'japurahéi', 'umi', 'polka', 'ha', 'guarania']
['B-MIX', 'B-GN', 'I-GN', 'I-GN', 'I-GN', 'I-GN', 'I-GN', 'B-FOREIGN', 'B-GN', 'B-ES']


In [91]:
from datasets import Dataset, DatasetDict

# Create Dataset objects from the parsed data
train_dataset = Dataset.from_dict(train_dataset_format)
dev_dataset = Dataset.from_dict(dev_dataset_format)
test_dataset = Dataset.from_dict(test_dataset_format)

# Create a DatasetDict
wnut = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset,
    "test": test_dataset
})

wnut

DatasetDict({
    train: Dataset({
        features: ['text', 'tokens', 'ner_tags'],
        num_rows: 1140
    })
    validation: Dataset({
        features: ['text', 'tokens', 'ner_tags'],
        num_rows: 180
    })
    test: Dataset({
        features: ['text', 'tokens', 'ner_tags'],
        num_rows: 180
    })
})

In [114]:
wnut.push_to_hub("mmaguero/gua-spa-2023-task-1-2")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########|  223kB /  223kB            

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 39.5kB / 39.5kB            

CommitInfo(commit_url='https://huggingface.co/datasets/mmaguero/gua-spa-2023-task-1-2/commit/fc363fa7f5ee4dd3e8cbac26dad728c1d1413791', commit_message='Upload dataset', commit_description='', oid='fc363fa7f5ee4dd3e8cbac26dad728c1d1413791', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mmaguero/gua-spa-2023-task-1-2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mmaguero/gua-spa-2023-task-1-2'), pr_revision=None, pr_num=None)

Then take a look at an example:

In [92]:
wnut["train"][0]

{'text': "Aldana he'íva umi kits ohóva ha oguahëva opavave temimbo'épe oñepyrû mboyve clase pero noñeguahëi mbohapýha ary ohóvape .",
 'tokens': ['Aldana',
  "he'íva",
  'umi',
  'kits',
  'ohóva',
  'ha',
  'oguahëva',
  'opavave',
  "temimbo'épe",
  'oñepyrû',
  'mboyve',
  'clase',
  'pero',
  'noñeguahëi',
  'mbohapýha',
  'ary',
  'ohóvape',
  '.'],
 'ner_tags': ['B-PER',
  'B-GN',
  'I-GN',
  'B-FOREIGN',
  'B-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'B-ES',
  'I-ES',
  'B-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'O']}

Each number in `ner_tags` represents an entity. Convert the numbers to their label names to find out what the entities are:

In [93]:
# Get the unique tag names from the 'ner_tags' column
unique_tags = set()
for tags_list in wnut["train"]["ner_tags"]:
    unique_tags.update(tags_list)

label_list = sorted(list(unique_tags),
                    key=lambda x: (x.split('-')[-1]))
label_list

['I-ES',
 'B-ES',
 'I-FOREIGN',
 'B-FOREIGN',
 'B-GN',
 'I-GN',
 'B-LOC',
 'I-LOC',
 'B-MIX',
 'I-MIX',
 'O',
 'B-ORG',
 'I-ORG',
 'B-PER',
 'I-PER']

The letter that prefixes each `ner_tag` indicates the token position of the entity:

- `B-` indicates the beginning of an entity.
- `I-` indicates a token is contained inside the same entity (for example, the `State` token is a part of an entity like
  `Empire State Building`).
- `0` indicates the token doesn't correspond to any entity.

In [94]:
def assing_ner_id(example):
    example["ner_tags_name"] = example["ner_tags"]
    example["ner_tags"] = []
    for tag in example["ner_tags_name"]:
        example["ner_tags"].append(label_list.index(tag))
    return example

wnut = wnut.map(assing_ner_id)
wnut["train"][0], wnut["test"][0], wnut["validation"][0]

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

({'text': "Aldana he'íva umi kits ohóva ha oguahëva opavave temimbo'épe oñepyrû mboyve clase pero noñeguahëi mbohapýha ary ohóvape .",
  'tokens': ['Aldana',
   "he'íva",
   'umi',
   'kits',
   'ohóva',
   'ha',
   'oguahëva',
   'opavave',
   "temimbo'épe",
   'oñepyrû',
   'mboyve',
   'clase',
   'pero',
   'noñeguahëi',
   'mbohapýha',
   'ary',
   'ohóvape',
   '.'],
  'ner_tags': [13, 4, 5, 3, 4, 5, 5, 5, 5, 5, 5, 1, 0, 4, 5, 5, 5, 10],
  'ner_tags_name': ['B-PER',
   'B-GN',
   'I-GN',
   'B-FOREIGN',
   'B-GN',
   'I-GN',
   'I-GN',
   'I-GN',
   'I-GN',
   'I-GN',
   'I-GN',
   'B-ES',
   'I-ES',
   'B-GN',
   'I-GN',
   'I-GN',
   'I-GN',
   'O']},
 {'text': "Igusto ñañe'ẽ guaranime ha avei japurahéi umi polka ha guarania",
  'tokens': ['Igusto',
   "ñañe'ẽ",
   'guaranime',
   'ha',
   'avei',
   'japurahéi',
   'umi',
   'polka',
   'ha',
   'guarania'],
  'ner_tags': [8, 4, 5, 5, 5, 5, 5, 3, 4, 1],
  'ner_tags_name': ['B-MIX',
   'B-GN',
   'I-GN',
   'I-GN',
   'I-GN',
 

In [95]:
ner = zip(wnut["train"][0]["tokens"], wnut["train"][0]["ner_tags"])
for item in ner:
  print(f"{item} => {label_list[item[1]]}")

('Aldana', 13) => B-PER
("he'íva", 4) => B-GN
('umi', 5) => I-GN
('kits', 3) => B-FOREIGN
('ohóva', 4) => B-GN
('ha', 5) => I-GN
('oguahëva', 5) => I-GN
('opavave', 5) => I-GN
("temimbo'épe", 5) => I-GN
('oñepyrû', 5) => I-GN
('mboyve', 5) => I-GN
('clase', 1) => B-ES
('pero', 0) => I-ES
('noñeguahëi', 4) => B-GN
('mbohapýha', 5) => I-GN
('ary', 5) => I-GN
('ohóvape', 5) => I-GN
('.', 10) => O


## Preprocess

In [14]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/iY2AZYdZAr0?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


The next step is to load a mBERT tokenizer to preprocess the `tokens` field:

In [96]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mmaguero/multilingual-bert-gn-base-cased") #"distilbert/distilbert-base-uncased")

As you saw in the example `tokens` field above, it looks like the input has already been tokenized. But the input actually hasn't been tokenized yet and you'll need to set `is_split_into_words=True` to tokenize the words into subwords. For example:

In [97]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'Al',
 '##dana',
 'he',
 "'",
 'í',
 '##va',
 'um',
 '##i',
 'kit',
 '##s',
 'o',
 '##h',
 '##ó',
 '##va',
 'ha',
 'og',
 '##uah',
 '##ë',
 '##va',
 'op',
 '##ava',
 '##ve',
 'temi',
 '##mbo',
 "'",
 'é',
 '##pe',
 'o',
 '##ñ',
 '##ep',
 '##yr',
 '##û',
 'm',
 '##boy',
 '##ve',
 'clase',
 'pero',
 'no',
 '##ñ',
 '##egu',
 '##ah',
 '##ë',
 '##i',
 'm',
 '##bo',
 '##ha',
 '##p',
 '##ý',
 '##ha',
 'ary',
 'o',
 '##h',
 '##ó',
 '##va',
 '##pe',
 '.',
 '[SEP]']

However, this adds some special tokens `[CLS]` and `[SEP]` and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the [`word_ids`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_ids) method.
2. Assigning the label `-100` to the special tokens `[CLS]` and `[SEP]` so they're ignored by the PyTorch loss function (see [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)).
3. Only labeling the first token of a given word. Assign `-100` to other subtokens from the same word.

Here is how you can create a function to realign the tokens and labels, and truncate sequences to be no longer than DistilBERT's maximum input length:

In [98]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [99]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [100]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [20]:
!pip install seqeval

In [101]:
import evaluate

seqeval = evaluate.load("seqeval")

Get the NER labels first, and then create a function that passes your true predictions and true labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the scores:

In [102]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [103]:
id2label = {
    0:'B-ES',
    1:'I-ES',
    2:'B-FOREIGN',
    3:'I-FOREIGN',
    4:'B-GN',
    5:'I-GN',
    6:'I-LOC',
    7:'B-LOC',
    8:'I-MIX',
    9:'B-MIX',
    10:'O',
    11:'B-ORG',
    12:'I-ORG',
    13:'I-PER',
    14:'B-PER'
}
label2id = {
    'B-ES':0,
    'I-ES':1,
    'B-FOREIGN':2,
    'I-FOREIGN':3,
    'B-GN':4,
    'I-GN':5,
    'I-LOC':6,
    'B-LOC':7,
    'I-MIX':8,
    'B-MIX':9,
    'O':10,
    'B-ORG':11,
    'I-ORG':12,
    'I-PER':13,
    'B-PER':14
}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForTokenClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForTokenClassification) along with the number of expected labels, and the label mappings:

In [24]:
#!pip uninstall -y transformers
#!pip install transformers[tf]

In [104]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "mmaguero/multilingual-bert-gn-base-cased", # "distilbert/distilbert-base-uncased",
    num_labels=15, id2label=id2label, label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at mmaguero/multilingual-bert-gn-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the seqeval scores and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [105]:
training_args = TrainingArguments(
    output_dir="langid-ner-multilingual-bert-gn-base-cased",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20, # 3 a 10 o más epochs
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit = 3,
    push_to_hub=False, # True para subir a HF
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.128789,0.500433,0.484899,0.492544,0.705253
2,No log,0.747609,0.598634,0.588087,0.593314,0.797926
3,No log,0.620277,0.632858,0.633389,0.633124,0.824356
4,No log,0.578372,0.641296,0.664430,0.652658,0.838742
5,No log,0.548720,0.662621,0.687081,0.674629,0.846437
6,No log,0.539558,0.692875,0.709732,0.701202,0.858147
7,0.692000,0.544646,0.691809,0.715604,0.703505,0.855470
8,0.692000,0.552776,0.693173,0.723993,0.708248,0.860154
9,0.692000,0.561405,0.698708,0.725671,0.711934,0.860154
10,0.692000,0.575105,0.707079,0.729027,0.717885,0.859819


/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.12/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=1440, training_loss=0.3227359692255656, metrics={'train_runtime': 1292.7209, 'train_samples_per_second': 17.637, 'train_steps_per_second': 1.114, 'total_flos': 1112505736786920.0, 'train_loss': 0.3227359692255656, 'epoch': 20.0})

In [106]:
trainer.evaluate(tokenized_wnut["test"])

{'eval_loss': 0.5980403423309326,
 'eval_precision': 0.7388748950461796,
 'eval_recall': 0.7463952502120441,
 'eval_f1': 0.7426160337552743,
 'eval_accuracy': 0.8673433671683585,
 'eval_runtime': 1.1215,
 'eval_samples_per_second': 160.494,
 'eval_steps_per_second': 10.7,
 'epoch': 20.0}

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [107]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...05355.1186bb0a8378.2670.0: 100%|##########| 11.0kB / 11.0kB            

  ...05851.1186bb0a8378.2670.1: 100%|##########|   560B /   560B            

  ...8154.1186bb0a8378.15041.0: 100%|##########| 21.2kB / 21.2kB            

...

  ...e-cased/model.safetensors:   0%|          |  570kB /  709MB            

  ...2457.1186bb0a8378.15041.2:  10%|#         | 1.61kB / 16.0kB            

  ...3751.1186bb0a8378.15041.3:  10%|#         |  56.0B /   560B            

  ...e-cased/training_args.bin:  10%|#         |   593B / 5.91kB            

CommitInfo(commit_url='https://huggingface.co/mmaguero/langid-ner-multilingual-bert-gn-base-cased/commit/b6659d1f56e9307910219abbd177a48d0fa1a0ca', commit_message='End of training', commit_description='', oid='b6659d1f56e9307910219abbd177a48d0fa1a0ca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mmaguero/langid-ner-multilingual-bert-gn-base-cased', endpoint='https://huggingface.co', repo_type='model', repo_id='mmaguero/langid-ner-multilingual-bert-gn-base-cased'), pr_revision=None, pr_num=None)

<Tip>

For a more in-depth example of how to finetune a model for token classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [131]:
text = "Golden State Warriors ha'e peteĩ basketball profesional aty USA pegua, oĩva San Francisco-pe."
text_tokens = text.split()

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for NER with your model, and pass your text to it:

In [143]:
from transformers import pipeline

classifier = pipeline("ner", model="mmaguero/langid-ner-multilingual-bert-gn-base-cased", aggregation_strategy="max")

Device set to use cuda:0


In [136]:
classifier(text)

[{'entity_group': 'ORG',
  'score': np.float32(0.48622647),
  'word': 'Golden State',
  'start': 0,
  'end': 12},
 {'entity_group': 'ES',
  'score': np.float32(0.42278388),
  'word': 'Warriors',
  'start': 13,
  'end': 21},
 {'entity_group': 'GN',
  'score': np.float32(0.99656606),
  'word': "ha ' e peteĩ",
  'start': 22,
  'end': 32},
 {'entity_group': 'ORG',
  'score': np.float32(0.68381894),
  'word': 'basketball',
  'start': 33,
  'end': 43},
 {'entity_group': 'ES',
  'score': np.float32(0.69726557),
  'word': 'profesional',
  'start': 44,
  'end': 55},
 {'entity_group': 'GN',
  'score': np.float32(0.9586266),
  'word': 'aty',
  'start': 56,
  'end': 59},
 {'entity_group': 'LOC',
  'score': np.float32(0.7862104),
  'word': 'USA',
  'start': 60,
  'end': 63},
 {'entity_group': 'GN',
  'score': np.float32(0.7438702),
  'word': 'pegua',
  'start': 64,
  'end': 69},
 {'entity_group': 'GN',
  'score': np.float32(0.9947167),
  'word': 'oĩva',
  'start': 71,
  'end': 75},
 {'entity_group'

In [111]:
wnut["test"][0]["tokens"], wnut["test"][0]["ner_tags"]

(['Igusto',
  "ñañe'ẽ",
  'guaranime',
  'ha',
  'avei',
  'japurahéi',
  'umi',
  'polka',
  'ha',
  'guarania'],
 [8, 4, 5, 5, 5, 5, 5, 3, 4, 1])

In [144]:
classifier(wnut["test"][0]["text"])

[{'entity_group': 'GN',
  'score': np.float32(0.97101665),
  'word': "Igusto ñañe ' ẽ guaranime ha avei japurahéi umi polka ha guarania",
  'start': 0,
  'end': 63}]

In [148]:
text = "Ko'ero aháta Paraguay-pe ha upéi ahásata Luque-pe che tío Kálo ogápe."
text_tokens = text.split()
classifier(text)

[{'entity_group': 'GN',
  'score': np.float32(0.9973824),
  'word': "Ko ' ero aháta",
  'start': 0,
  'end': 12},
 {'entity_group': 'LOC',
  'score': np.float32(0.9614328),
  'word': 'Paraguay',
  'start': 13,
  'end': 21},
 {'entity_group': 'GN',
  'score': np.float32(0.49895406),
  'word': '-',
  'start': 21,
  'end': 22},
 {'entity_group': 'GN',
  'score': np.float32(0.98736084),
  'word': 'pe',
  'start': 22,
  'end': 24},
 {'entity_group': 'GN',
  'score': np.float32(0.99493927),
  'word': 'ha upéi ahásata',
  'start': 25,
  'end': 40},
 {'entity_group': 'LOC',
  'score': np.float32(0.9406095),
  'word': 'Luque',
  'start': 41,
  'end': 46},
 {'entity_group': 'GN',
  'score': np.float32(0.89259255),
  'word': '-',
  'start': 46,
  'end': 47},
 {'entity_group': 'GN',
  'score': np.float32(0.9891623),
  'word': 'pe',
  'start': 47,
  'end': 49},
 {'entity_group': 'GN',
  'score': np.float32(0.9815431),
  'word': 'che',
  'start': 50,
  'end': 53},
 {'entity_group': 'ES',
  'score': 

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return PyTorch tensors:

In [36]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mmaguero/langid-ner-multilingual-bert-gn-base-cased")
inputs = tokenizer(text, return_tensors="pt")

Pass your inputs to the model and return the `logits`:

In [37]:
import torch
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("mmaguero/langid-ner-multilingual-bert-gn-base-cased")
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [38]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
inputs, predicted_token_class

({'input_ids': tensor([[  101, 30186,   112, 10163, 10133, 69863, 86367, 25118,   118, 11161,
          10228, 10741, 10333, 69863, 17082, 10213, 23859, 11189,   118, 11161,
          10262, 94354,   148, 14205, 10133, 10156, 77588, 10112,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1]])},
 ['I-LOC',
  'B-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'I-LOC',
  'B-GN',
  'B-GN',
  'B-GN',
  'I-GN',
  'I-GN',
  'I-GN',
  'I-MIX',
  'I-GN',
  'I-LOC',
  'B-LOC',
  'B-GN',
  'B-GN',
  'B-GN',
  'I-ES',
  'B-LOC',
  'I-ORG',
  'B-LOC',
  'B-GN',
  'B-GN',
  'I-GN',
  'O',
  'I-LOC'])

# Task
Push the `wnut` DatasetDict object to the Hugging Face Hub.

## Push dataset to Hugging Face Hub

### Subtask:
Push the `wnut` DatasetDict object to the Hugging Face Hub.


## Summary:

### Insights or Next Steps
*   The next step is to implement the functionality to push the `wnut` DatasetDict object to the Hugging Face Hub, completing the data sharing task.
